<h2>Introduction/Business Problem</h2>

<p>
    This study supposes owner of a local coffee shop sucessfully running the shop located along train line around central of Tokyo, Japan.<br>
    The owner is considering to open the second shop and needs to find the right place for the second shop to do great business.<br> 
    The owners strategies to find the place is this.<br>
    <ul>
        <li>Similar environment to the first shop so as to utilize know-how obtained through the experience in the first shop.</li>
        <li>Along same train line to the first shop.</li>
    </ul>
    In this report, the right place for the second shop are studied with data analysis methodology.
</p>

<h2>Data</h2>

<p>
    Set of latitude and longtitude of stations around central of Tokyo are used for this study.<br>
    Targeted lines are following 2 lines which located around central of Tokyo.
    <ul>
        <li>Keihin-Tohoku Line</li>
        <li>Negishi Line</li>
    </ul>
    Latitude and longtitude are collected from the link below and merged into csv file('train_station_around_Tokyo.csv') for this study.<br>
    (Name of station are translated from Japanese into English manually so that they can be read.)<br>
    <br>
    <a href="https://www.odekakemap.com/">https://www.odekakemap.com/</a><br>
    <br>
    The nearest station from the first shop is <b>Akabane</b> station in Keihin-Tohoku Line<br>
</p>

<h2>Methodology</h2>

<p>
    The main processes for the study are as follows.
    <ol>
        <li>Obtaining the shop information around each station in the targeted train line via FourSquare API</li>
        <li>According to the shop information, clustering of the stations are execuded via machine-learning algorithm.</li>
        <li>Based on the result of clustering, find similar stations to the nearest station from the first shop.</li>
    </ol>
    Clustering via machine-learning are conducted to find the place similar to the place where the 1st shop is located.<br>
    K-means algorithm are employeed for the clustering because it is very common for clustering via machine-leaning.<br>
</p>

<h2>Result</h2>

<h3>Importing libraries</h3>

In [1]:
import pandas as pd
import numpy as np

<h3>data import</h3>

In [2]:
df=pd.read_csv('train_station_around_Tokyo.csv')
df.head()

,Station,Latitude,Longitude
0,Omiya,35.906295,139.623999
1,Saitama-Shintoshin,35.893867,139.633587
2,Yono,35.884393,139.639085
3,Kita-Urawa,35.872156,139.646196
4,Urawa,35.858482,139.657087


<h3>Importing libraries</h3>

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


<h3>setting for foursquare API</h3>

In [4]:
CLIENT_ID = 'MOA1B3H2LU30ONSQ4DH5ATKS03OBOUYJUDKZVU4LPAD1Q0WQ' # your Foursquare ID
CLIENT_SECRET = 'ZCYRMC0VEYR1E5MR4UKASYEXECH2IRRS5APEBWC1AXXXXIQH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 300 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MOA1B3H2LU30ONSQ4DH5ATKS03OBOUYJUDKZVU4LPAD1Q0WQ
CLIENT_SECRET:ZCYRMC0VEYR1E5MR4UKASYEXECH2IRRS5APEBWC1AXXXXIQH


<h3>Defining functions for analysis</h3>

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3>Searching venues around neighborhoods and data arrangement for clustering</h3>

In [8]:
venues = getNearbyVenues(names=df['Station'],
                         latitudes=df['Latitude'],
                         longitudes=df['Longitude']
                         )

Omiya
Saitama-Shintoshin
Yono
Kita-Urawa
Urawa
Minami-Urawa
Warabi
Nishi-Kawaguchi
Kawaguchi
Akabane
Higashi-Jujo
Oji
Kami-Nakazato
Tabata
Nishi-Nippori
Nippori
Uguisudani
Ueno
Okachimachi
Akihabara
Kanda
Tokyo
Yurakucho
Shimbashi
Hamamatsucho
Tamachi
Shinagawa
Oimachi
Omori
Kamata
Kawasaki
Tsurumi
Shin-Koyasu
Higashi-Kanagawa
Yokohama
Sakuragicho
Kannai
Ishikawacho
Yamate
Negishi
Isogo
Shin-Sugita
Yokodai
Konandai
Hongodai
Ofuna


In [9]:
print(venues.shape)
venues.head()

(2940, 7)


,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Omiya,35.906295,139.623999,酒蔵 力 大宮東口駅前店,35.906930,139.625562,Sake Bar
1,Omiya,35.906295,139.623999,メイドカフェ HoneyHoney,35.907286,139.625989,Café
2,Omiya,35.906295,139.623999,BAKE CHEESE TART,35.906989,139.623819,Dessert Shop
3,Omiya,35.906295,139.623999,つくば本店 駅前店,35.906792,139.625258,Noodle House
4,Omiya,35.906295,139.623999,Mouko Tanmen Nakamoto (蒙古タンメン中本),35.905107,139.625840,Ramen Restaurant


In [10]:
venues.groupby('Station').count()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Station,,,,,,
Akabane,100,100,100,100,100,100
Akihabara,100,100,100,100,100,100
Hamamatsucho,71,71,71,71,71,71
Higashi-Jujo,44,44,44,44,44,44
Higashi-Kanagawa,64,64,64,64,64,64
Hongodai,15,15,15,15,15,15
Ishikawacho,25,25,25,25,25,25
Isogo,16,16,16,16,16,16
Kamata,72,72,72,72,72,72


In [11]:
# one hot encoding
df_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Station'] = venues['Station'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Station,ATM,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Center,Butcher,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Box,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Paper / Office Supplies Store,Park,Pastry Shop,Pet Café,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soba Restaurant,Soccer Field,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Toll Booth,Tonkatsu Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [12]:
df_grouped = df_onehot.groupby('Station').mean().reset_index()
df_grouped

,Station,ATM,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Center,Butcher,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Box,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Paper / Office Supplies Store,Park,Pastry Shop,Pet Café,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soba Restaurant,Soccer Field,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Toll Booth,Tonkatsu Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Akabane,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.040000,0.00,0.01,0.010000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.090000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000

In [13]:
num_top_venues = 5

for hood in df_grouped['Station']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Station'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akabane----
               venue  freq
0           Sake Bar  0.18
1  Convenience Store  0.09
2   Ramen Restaurant  0.06
3          BBQ Joint  0.04
4    Soba Restaurant  0.04


----Akihabara----
               venue  freq
0           Sake Bar  0.12
1         Hobby Shop  0.10
2               Café  0.05
3             Arcade  0.04
4  Electronics Store  0.04


----Hamamatsucho----
                 venue  freq
0            BBQ Joint  0.10
1    Convenience Store  0.08
2     Ramen Restaurant  0.07
3          Coffee Shop  0.06
4  Japanese Restaurant  0.06


----Higashi-Jujo----
                venue  freq
0   Convenience Store  0.16
1            Sake Bar  0.14
2       Shopping Mall  0.05
3         Pizza Place  0.05
4  Chinese Restaurant  0.05


----Higashi-Kanagawa----
                venue  freq
0   Convenience Store  0.22
1    Ramen Restaurant  0.11
2  Chinese Restaurant  0.06
3         Coffee Shop  0.05
4         Supermarket  0.05


----Hongodai----
               venue  freq
0  Convenie

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
station_venues_sorted = pd.DataFrame(columns=columns)
station_venues_sorted['Station'] = df_grouped['Station']

for ind in np.arange(df_grouped.shape[0]):
    station_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

station_venues_sorted.head()

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akabane,Sake Bar,Convenience Store,Ramen Restaurant,Soba Restaurant,BBQ Joint,Italian Restaurant,Coffee Shop,Bar,Dumpling Restaurant,Shopping Mall
1,Akihabara,Sake Bar,Hobby Shop,Café,Electronics Store,Arcade,Dessert Shop,Ramen Restaurant,Bar,Japanese Curry Restaurant,Steakhouse
2,Hamamatsucho,BBQ Joint,Convenience Store,Ramen Restaurant,Japanese Restaurant,Soba Restaurant,Coffee Shop,Yoshoku Restaurant,Tonkatsu Restaurant,Seafood Restaurant,Chinese Restaurant
3,Higashi-Jujo,Convenience Store,Sake Bar,Ramen Restaurant,Supermarket,Chinese Restaurant,Tonkatsu Restaurant,Pizza Place,Shopping Mall,Discount Store,Wagashi Place
4,Higashi-Kanagawa,Convenience Store,Ramen Restaurant,Chinese Restaurant,Supermarket,Coffee Shop,Park,Japanese Restaurant,Soba Restaurant,Deli / Bodega,Sushi Restaurant


<h3>Building model for clustering by k-means</h3>

In [16]:
# set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 3, 4, 0, 2, 3, 0, 1, 0])

In [17]:
# add clustering labels
station_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(station_venues_sorted.set_index('Station'), on='Station')

df_merged.head() # check the last columns!

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Omiya,35.906295,139.623999,1,Sake Bar,Ramen Restaurant,Café,Convenience Store,Japanese Restaurant,Bookstore,Coffee Shop,Steakhouse,Bar,BBQ Joint
1,Saitama-Shintoshin,35.893867,139.633587,3,Coffee Shop,Convenience Store,Café,Shopping Mall,Italian Restaurant,Burger Joint,Chinese Restaurant,Clothing Store,Plaza,Ramen Restaurant
2,Yono,35.884393,139.639085,2,Convenience Store,Supermarket,Japanese Restaurant,Dumpling Restaurant,Coffee Shop,Sake Bar,Donburi Restaurant,Italian Restaurant,Spa,Park
3,Kita-Urawa,35.872156,139.646196,0,Convenience Store,Ramen Restaurant,Café,Fast Food Restaurant,Soba Restaurant,Donburi Restaurant,ATM,Chinese Restaurant,Sake Bar,Record Shop
4,Urawa,35.858482,139.657087,3,Coffee Shop,Café,Bookstore,Italian Restaurant,Unagi Restaurant,Sake Bar,Ramen Restaurant,Supermarket,Convenience Store,Intersection


<h3>Indicating result of clustering on the map</h3>

In [18]:
latitude = df['Latitude'].mean()
longitude = df['Longitude'].mean()
print('The geograpical coordinate of the center of map is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the center of map is 35.63254521739131, 139.69069850000002.


In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#colors_array = cm.rainbow(np.linspace(0, 1, len(x)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Station'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examining each cluster</h3>

In [20]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[0] + list(range(4, df_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Kita-Urawa,Convenience Store,Ramen Restaurant,Café,Fast Food Restaurant,Soba Restaurant,Donburi Restaurant,ATM,Chinese Restaurant,Sake Bar,Record Shop
6,Warabi,Convenience Store,Ramen Restaurant,Café,Donburi Restaurant,Noodle House,Restaurant,Japanese Restaurant,Chinese Restaurant,Pharmacy,Discount Store
7,Nishi-Kawaguchi,Convenience Store,Ramen Restaurant,Park,Chinese Restaurant,ATM,BBQ Joint,Donburi Restaurant,Drugstore,Rock Club,Sake Bar
12,Kami-Nakazato,Convenience Store,Chinese Restaurant,Train Station,Science Museum,Park,Gym,Candy Store,Bath House,Platform,Historic Site
32,Shin-Koyasu,Convenience Store,Sake Bar,Office,Historic Site,Canal,Shipping Store,Gym / Fitness Center,Chinese Restaurant,Café,Train Station
33,Higashi-Kanagawa,Convenience Store,Ramen Restaurant,Chinese Restaurant,Supermarket,Coffee Shop,Park,Japanese Restaurant,Soba Restaurant,Deli / Bodega,Sushi Restaurant
38,Yamate,Convenience Store,ATM,Bakery,Ramen Restaurant,Café,Park,Sports Club,Trail,Train Station,Supermarket
39,Negishi,Convenience Store,Supermarket,Pool,Chinese Restaurant,Train Station,Pachinko Parlor,Bakery,Donut Shop,History Museum,Coffee Shop
40,Isogo,Convenience Store,Chinese Restaurant,Photography Studio,Fishing Store,Ramen Restaurant,Discount Store,Toy / Game Store,Fried Chicken Joint,Cosmetics Shop,Japanese Restaurant


In [21]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[0] + list(range(4, df_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Omiya,Sake Bar,Ramen Restaurant,Café,Convenience Store,Japanese Restaurant,Bookstore,Coffee Shop,Steakhouse,Bar,BBQ Joint
15,Nippori,Japanese Restaurant,Ramen Restaurant,Bed & Breakfast,Convenience Store,Soba Restaurant,Sake Bar,Fabric Shop,Noodle House,Café,Gift Shop
17,Ueno,Sake Bar,Wagashi Place,Japanese Restaurant,Chinese Restaurant,Café,Coffee Shop,Cocktail Bar,Science Museum,Ramen Restaurant,Convenience Store
18,Okachimachi,Sake Bar,BBQ Joint,Tonkatsu Restaurant,Ramen Restaurant,Chinese Restaurant,Pizza Place,Wagashi Place,Café,Yoshoku Restaurant,Hotel
19,Akihabara,Sake Bar,Hobby Shop,Café,Electronics Store,Arcade,Dessert Shop,Ramen Restaurant,Bar,Japanese Curry Restaurant,Steakhouse
20,Kanda,Ramen Restaurant,Bed & Breakfast,Japanese Restaurant,Sake Bar,Beer Bar,Japanese Curry Restaurant,Unagi Restaurant,Yoshoku Restaurant,Soba Restaurant,Café
22,Yurakucho,Café,Japanese Restaurant,Italian Restaurant,Bakery,Chinese Restaurant,Steakhouse,Gourmet Shop,Boutique,Clothing Store,French Restaurant
23,Shimbashi,Ramen Restaurant,Sake Bar,Japanese Restaurant,BBQ Joint,Hotel,Tonkatsu Restaurant,Italian Restaurant,French Restaurant,Sushi Restaurant,Hotel Bar
25,Tamachi,Japanese Restaurant,Italian Restaurant,Coffee Shop,Chinese Restaurant,BBQ Joint,Sake Bar,Soba Restaurant,Bar,Ramen Restaurant,Bistro
26,Shinagawa,Sake Bar,Japanese Restaurant,Bar,Bakery,Coffee Shop,Steakhouse,Hotel,BBQ Joint,Café,Seafood Restaurant


In [22]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[0] + list(range(4, df_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Yono,Convenience Store,Supermarket,Japanese Restaurant,Dumpling Restaurant,Coffee Shop,Sake Bar,Donburi Restaurant,Italian Restaurant,Spa,Park
5,Minami-Urawa,Convenience Store,Sake Bar,Park,Supermarket,Train Station,Japanese Restaurant,Chinese Restaurant,Deli / Bodega,Ramen Restaurant,Italian Restaurant
42,Yokodai,Convenience Store,Deli / Bodega,Supermarket,Intersection,Grocery Store,BBQ Joint,Steakhouse,Train Station,Noodle House,Bus Station
43,Konandai,Convenience Store,Coffee Shop,Donburi Restaurant,Sake Bar,Fast Food Restaurant,Bookstore,Restaurant,Clothing Store,Electronics Store,Chinese Restaurant
44,Hongodai,Convenience Store,Supermarket,Coffee Shop,Mini Golf,Train Station,Fast Food Restaurant,Shoe Store,Bus Station,Plaza,Farmers Market
45,Ofuna,Convenience Store,Ramen Restaurant,Japanese Restaurant,Sake Bar,Italian Restaurant,Train Station,Chinese Restaurant,Coffee Shop,BBQ Joint,Discount Store


In [23]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[0] + list(range(4, df_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Saitama-Shintoshin,Coffee Shop,Convenience Store,Café,Shopping Mall,Italian Restaurant,Burger Joint,Chinese Restaurant,Clothing Store,Plaza,Ramen Restaurant
4,Urawa,Coffee Shop,Café,Bookstore,Italian Restaurant,Unagi Restaurant,Sake Bar,Ramen Restaurant,Supermarket,Convenience Store,Intersection
8,Kawaguchi,Ramen Restaurant,Coffee Shop,Fast Food Restaurant,Convenience Store,Clothing Store,Supermarket,Park,Rock Club,Drugstore,Shopping Mall
13,Tabata,Convenience Store,Sake Bar,Japanese Restaurant,Chinese Restaurant,Soba Restaurant,Coffee Shop,Park,Train Station,Grocery Store,Udon Restaurant
21,Tokyo,Café,Hotel,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Deli / Bodega,Train Station,Hobby Shop,Bakery,Building
24,Hamamatsucho,BBQ Joint,Convenience Store,Ramen Restaurant,Japanese Restaurant,Soba Restaurant,Coffee Shop,Yoshoku Restaurant,Tonkatsu Restaurant,Seafood Restaurant,Chinese Restaurant
36,Kannai,Convenience Store,Hotel,Baseball Stadium,Ramen Restaurant,Indian Restaurant,Café,Rock Club,Coffee Shop,Bed & Breakfast,Soba Restaurant
37,Ishikawacho,Italian Restaurant,Café,Historic Site,Dumpling Restaurant,Garden,Ramen Restaurant,Supermarket,Park,Tea Room,Shopping Mall


In [24]:
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[0] + list(range(4, df_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Akabane,Sake Bar,Convenience Store,Ramen Restaurant,Soba Restaurant,BBQ Joint,Italian Restaurant,Coffee Shop,Bar,Dumpling Restaurant,Shopping Mall
10,Higashi-Jujo,Convenience Store,Sake Bar,Ramen Restaurant,Supermarket,Chinese Restaurant,Tonkatsu Restaurant,Pizza Place,Shopping Mall,Discount Store,Wagashi Place
11,Oji,Convenience Store,Sake Bar,Ramen Restaurant,Museum,Grocery Store,Park,Japanese Restaurant,Theater,History Museum,Drugstore
14,Nishi-Nippori,Convenience Store,Ramen Restaurant,Japanese Restaurant,Sake Bar,BBQ Joint,Udon Restaurant,Yakitori Restaurant,Diner,Soba Restaurant,Shopping Mall
16,Uguisudani,Convenience Store,Sake Bar,History Museum,Ramen Restaurant,Japanese Restaurant,Coffee Shop,Grocery Store,Museum,Hostel,BBQ Joint
28,Omori,Convenience Store,Sake Bar,Coffee Shop,Ramen Restaurant,BBQ Joint,Indian Restaurant,Discount Store,Café,Japanese Restaurant,ATM
31,Tsurumi,Ramen Restaurant,Convenience Store,Supermarket,Sake Bar,Italian Restaurant,Bakery,Bed & Breakfast,Bus Stop,Bookstore,Mobile Phone Shop
41,Shin-Sugita,Convenience Store,Ramen Restaurant,Sake Bar,BBQ Joint,Chinese Restaurant,Supermarket,Coffee Shop,Train Station,Donburi Restaurant,Noodle House


<p></p>

<h2>Discussion</h2>

<p>
    Now, it was found that <b>Akebane</b> station which is the nearest station from the first shop labeled cluster<b>4</b>; therefore, it was suggested that stations labeled cluste3 are possible candidate for the second shop place according to K-means clustering.<br>
    For mode detail analysis, similarity of the stations labeled cluster3 to <b>Akabane</b> station by Euclidean distance which is used in K-means algorithm.
</p>

In [25]:
#Extract stations labeled cluster4
df_grouped_with_label=pd.concat([df_grouped,pd.DataFrame(kmeans.labels_,columns=['Cluster Labels'])],axis=1)
df_grouped_with_label=pd.concat([df_grouped_with_label.iloc[:,[0]],
                                 df_grouped_with_label.iloc[:,[len(df_grouped_with_label.columns)-1]],
                                 df_grouped_with_label.iloc[:,1:(len(df_grouped_with_label.columns)-1)]],axis=1)
df_grouped_cluster4=df_grouped_with_label[df_grouped_with_label['Cluster Labels']==4]
df_grouped_cluster4

,Station,Cluster Labels,ATM,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Center,Butcher,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Box,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Paper / Office Supplies Store,Park,Pastry Shop,Pet Café,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soba Restaurant,Soccer Field,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Toll Booth,Tonkatsu Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Akabane,4,0.010000,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040000,0.0,0.01,0.010000,0.03,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.01,0.010000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.020000,0.0,0.0,0.01,0.0,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.0,0.0,0.000000,0.090000,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,0.0,0.000000,0.020000,0.010000,0.0,0.000000,0.000000,0.020000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.010000,0.01,0.0,0.0,0.0,0.00,0.010000,0.000000,0.000000,0.0,0.00

In [26]:
#define the function to calculate euclidean distance to Akabane station
def Akabane_calc(vec):
    vec=vec[2:]
    return np.linalg.norm(df_grouped_cluster4.iloc[0,2:]-vec)

In [27]:
#Calculate euclidean distance and merged with dataframe
euc_dist_from_akabane=pd.DataFrame(df_grouped_cluster4.apply(Akabane_calc,axis=1),columns=['Euclidean distance from Akabane'])
df_grouped_cluster4=pd.concat([df_grouped_cluster4,euc_dist_from_akabane],axis=1)
df_grouped_cluster4.sort_values('Euclidean distance from Akabane',ascending=True)

,Station,Cluster Labels,ATM,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Center,Butcher,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Fishing Store,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Box,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Pachinko Parlor,Paper / Office Supplies Store,Park,Pastry Shop,Pet Café,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soba Restaurant,Soccer Field,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Toll Booth,Tonkatsu Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant,Euclidean distance from Akabane
0,Akabane,4,0.010000,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040000,0.0,0.01,0.010000,0.03,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.01,0.010000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.020000,0.0,0.0,0.01,0.0,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.0,0.0,0.000000,0.090000,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,0.0,0.000000,0.020000,0.010000,0.0,0.000000,0.000000,0.020000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.010000,0.01,0.0,0.0,0.0,0.00,0.0

<p>According to Euclidean distance, it was suggested that best place for the second shop would be <b>Omori</b> station.</p>

<h2>Conclusion</h2>

<p>
    Trough the data analysis with K-means clustering, following ideas were suggested.
    <ul>
        <li>Stations labeled cluster4 by k-means clustering will be possibe candidate for the second shop place.</li>
        <li>Especially, <b>Omori</b> station would be the best place.</li>
    </ul>
</p>